In [1]:
"""
Load Lenet model in GPU

"""

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
import time

def binary_transform(image):
    return (image > 0).float()

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 120, 5)
        self.fc1 = nn.Linear(120, 84)
        self.fc2 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.pool1(self.conv1(x)))
        x = torch.relu(self.pool2(self.conv2(x)))
        x = torch.relu(self.conv3(x))
        x = x.view(-1, 120 * 1 * 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loaded_model = torch.load('lenet.pth')
loaded_model.to(device)  

test_transform = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
    binary_transform
    # transforms.Normalize((0.1307,), (0.3081,))
])
test_transform_original = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
    # transforms.Normalize((0.1307,), (0.3081,))
])

test_dataset_original = datasets.MNIST('./data', train=False, download=True, transform=test_transform_original)
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=test_transform)


test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)
test_loader_original = torch.utils.data.DataLoader(test_dataset_original, batch_size=10000, shuffle=False)

correct = 0
total = 0


with torch.no_grad():
    for data, targets in test_loader:
        data = data.to(device)  
        targets = targets.to(device)
        outputs = loaded_model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()


print(f'Accuracy on binary test set: {100 * correct / total}%')

correct = 0
total = 0

with torch.no_grad():
    for data, targets in test_loader_original:
        data = data.to(device)
        targets = targets.to(device)
        outputs = loaded_model(data)
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += (predicted == targets).sum().item()

print(f'Accuracy on original test set: {100 * correct / total}%')



Accuracy on binary test set: 98.3%
Accuracy on original test set: 98.95%


In [1]:
! pip install pynvml

     -------------------------------------- 53.1/53.1 kB 119.2 kB/s eta 0:00:00


In [ ]:
"""
Calculate the time and power consumption corresponding to inferring an image on the GPU

"""
import pynvml
pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)
total_time = 0;
count=10000
for i in range (count):

    data_original, target_original = test_dataset_original[i]
    data_tensor = data_original.to(device)

    start_time = time.perf_counter_ns()
    outputs =loaded_model(data_tensor)
    end_time = time.perf_counter_ns()

    execution_time = end_time - start_time
    total_time += execution_time

print(device)
print(count)
print(f"One inference time：{total_time/count} ns")
powerusage = pynvml.nvmlDeviceGetPowerUsage(handle) / 1000
print(f"GPU Power Usage: {powerusage} w")